In [1]:
################################################################################################
# Without fine-tuning of the pretrained neural network. 
################################################################################################


###############################
#  cla = '1' or '2' response: speed  > 200
###############################
cla = '1'

In [2]:
import numpy as np
import pandas as pd
import os
import glob
from PIL import Image
import torch
import torchvision.transforms as transforms
import torchvision 
from torch.utils import data as D
import pickle
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from util import *

In [3]:
######################################################################################################################################################
# preparation.
######################################################################################################################################################

# ############################################################################################
# # Extract directory addresses of the img and label files
# ############################################################################################
class DConfig(object):
    parent_folder = './data/compcars/data/label/'
    attri = './data/compcars/data/misc/attributes.txt'

op = DConfig()
# obtain the file addressess in a folder
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

In [4]:
# ############################################################################################
# # Read the attribute file and drop the lines with missing 'speed'
# ############################################################################################
attriArray = np.empty((0, 6))
with open(op.attri, newline='\n') as trainfile:
    for line in trainfile:
        line3 = line.replace("\n","")
        line4 = line3.split(' ')
        attriArray = np.concatenate((attriArray, np.array(line4).reshape(1,-1)), axis = 0)

attriArray = attriArray[1:,:]


In [5]:
########################
# Read the stored dataset
########################
infile = open('subsetRes4.p', 'rb')
subsetRes = pickle.load(infile)
infile.close()

labelArray_label1 = subsetRes['labelArray_label1']
labelArray_label3 = subsetRes['labelArray_label3']
labelArray_label2 = subsetRes['labelArray_label2']


if cla == '1':
    labelArray4 = labelArray_label1
elif cla == '3':
    labelArray4 = labelArray_label3
elif cla == '2':
    labelArray4 = labelArray_label2
else:
    print('error')
    
########################
# the response label is whether the max speed of the car is larger than 'sped'
########################
sped = 200

group1Ind = np.where(labelArray4[:,5] > sped)[0]
group2Ind = np.where(labelArray4[:,5] <= sped)[0]

########################
# split the image indices into training and validation
########################
np.random.seed(10)
shuffleInd1 = np.arange(group1Ind.shape[0])
np.random.shuffle(shuffleInd1)

shuffleInd2 = np.arange(group2Ind.shape[0])
np.random.shuffle(shuffleInd2)

halfNum1 = round(shuffleInd1.shape[0]/2)
group1IndTrain, group1IndVal = group1Ind[shuffleInd1[:halfNum1]], group1Ind[shuffleInd1[halfNum1]:]

halfNum2 = round(shuffleInd2.shape[0]/2)
group2IndTrain, group2IndVal = group2Ind[shuffleInd2[:halfNum2]], group2Ind[shuffleInd2[halfNum2]:]

# combine the splitted indices for response = 1 and response = 0
TrainInd = np.concatenate((group1IndTrain, group2IndTrain))
ValInd = np.concatenate((group1IndVal, group2IndVal))

In [6]:
###############################
#  Define the dataset object. Apply necessary transform for alexnet
###############################
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
    	# new trial
    	# transforms.RandomPerspective(),
        # classical
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),

        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
class CompcarsDS(D.Dataset):
    """
    A customized data loader.
    """
    def __init__(self, Ind, mode):
        """ Intialize the dataset
        """

        self.filenames = labelArray4[Ind,3]

        self.len = len(self.filenames)
        self.transform = data_transforms[mode]
                           
    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        image = Image.open(self.filenames[index])
        model_id = self.filenames[index].split("/")[6]
        # obtain the car speed
        label = int(int(attriArray[np.where(attriArray[:,0] == model_id),1][0,0]) > sped)
        return self.transform(image), label
    
    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len


# prepare train val datasets and data loaders
datTrain = CompcarsDS(Ind = TrainInd, mode = 'train')
datVal= CompcarsDS(Ind = ValInd, mode = 'val')
image_datasets = {'train': datTrain, 'val': datVal}

batch_size = 6

# NOTE: shuffle has to be true to have a balanced transfer learning!!
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print('dataset_sizes: ', dataset_sizes)

dataset_sizes:  {'train': 1090, 'val': 1875}


In [9]:
###########################################
# define feature extraction model
##########################################

d = 15 # the final output dim, so the architecture for mobilenetv2 is 1280 - d (retrain) - 2 (retrain)

model = torchvision.models.mobilenet_v2(pretrained=True) #used for cat-dog

print(model)

for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(model.last_channel, d),
        )
model.fc2 = nn.Linear(d, 2)




def extract_feature(dataloader, model, d, savefile, device='cpu'):
# set model to evaluation mode
    model.eval()  
    model.to(device)

    '''
		extract features in the form of yX from dataloader and model
	'''

    X = np.zeros((0, d))
    y = np.zeros(0)

    with torch.no_grad():  # disable gradient computation
        for i, (inputs, labels) in enumerate(dataloader):
            print('batch', i)
            inputs = inputs.to(device)

            # Forward pass to extract features
            f = model(inputs)

            batchsize = labels.size(0)
            f = f.view(batchsize, -1).cpu().numpy()  # move to CPU before numpy
            labels = labels.numpy()

            X = np.concatenate((X, f), axis=0)
            y = np.concatenate((y, labels), axis=0)

    yX = np.concatenate((y.reshape((-1,1)), X), axis=1)
    print('responses and features \n', yX)
    print('size:', yX.shape)
    np.savetxt(savefile, yX, delimiter=",")




MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [8]:
outputTrain = CompcarsDS(Ind = TrainInd, mode = 'val')
outputVal = CompcarsDS(Ind = ValInd, mode = 'val')

dataloaderTrain = torch.utils.data.DataLoader(outputTrain, batch_size=batch_size, shuffle=False)
dataloaderVal = torch.utils.data.DataLoader(outputVal, batch_size=batch_size, shuffle=False)

extract_feature(dataloaderTrain, model, d, "TrainSpeed_" + cla + "without_finetune.csv")
extract_feature(dataloaderVal, model, d, "ValSpeed_" + cla + "without_finetune.csv")

batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16
batch 17
batch 18
batch 19
batch 20
batch 21
batch 22
batch 23
batch 24
batch 25
batch 26
batch 27
batch 28
batch 29
batch 30
batch 31
batch 32
batch 33
batch 34
batch 35
batch 36
batch 37
batch 38
batch 39
batch 40
batch 41
batch 42
batch 43
batch 44
batch 45
batch 46
batch 47
batch 48
batch 49
batch 50
batch 51
batch 52
batch 53
batch 54
batch 55
batch 56
batch 57
batch 58
batch 59
batch 60
batch 61
batch 62
batch 63
batch 64
batch 65
batch 66
batch 67
batch 68
batch 69
batch 70
batch 71
batch 72
batch 73
batch 74
batch 75
batch 76
batch 77
batch 78
batch 79
batch 80
batch 81
batch 82
batch 83
batch 84
batch 85
batch 86
batch 87
batch 88
batch 89
batch 90
batch 91
batch 92
batch 93
batch 94
batch 95
batch 96
batch 97
batch 98
batch 99
batch 100
batch 101
batch 102
batch 103
batch 104
batch 105
batch 106
batch 107
batch 108
batch 109
batch 110
